In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['COT']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [14]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0.5]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [15]:
list(zip(longforms, counts))

[('carnitine octanoyltransferase', 11),
 ('cotinine', 8),
 ('conventional oxygen therapy', 8),
 ('cyclooctatetraene', 8),
 ('cotyledonary', 5),
 ('cotransport', 4),
 ('celastrus orbiculatus thunb', 4),
 ('chronic opioid therapy', 4),
 ('compliance over time', 3),
 ('cancer osaka thyroid', 2),
 ('casualty operation theater', 2),
 ('clinical operation tolerance', 2),
 ('cleft of trimer', 2),
 ('claudication onset time', 2),
 ('conventional oxygen treatment', 2),
 ('convulsive therapy', 2),
 ('choice of antibiotic treatment', 2),
 ('combination', 2),
 ('cancer ovarian tissues', 1),
 ('carnitine octanoyl transferase', 1),
 ('carotico oculomotor triangle', 1),
 ('celastrus orbiculatus thumb', 1),
 ('cloud optical thickness', 1),
 ('co transport', 1),
 ('cotyledonay', 1),
 ('committee on trauma', 1),
 ('cost of transport', 1),
 ('choice of treatment', 1),
 ('chicken ovotransferrin', 1),
 ('celastrus orbiculatus thunb extract', 1),
 ('the correlations', 1),
 ('compare to standard therapy', 1)

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [39]:
grounding_map, names, pos_labels = ({'carnitine octanoyltransferase': 'HGNC:6860',
  'celastrus orbiculatus thunb': 'ungrounded',
  'chronic opioid therapy': 'ungrounded',
  'cleft of trimer': 'ungrounded',
  'compliance over time': 'ungrounded',
  'conventional oxygen therapy': 'ungrounded',
  'cotinine': 'MESH:D003367',
  'cotyledonary': 'ungrounded',
  'cyclooctatetraene': 'CHEBI:CHEBI:47034',
  'k cl cotransport': 'ungrounded',
  'map3k8': 'HGNC:6860',
  'kinase 8': 'HGNC:6860'},
 {'HGNC:6860': 'MAP3K8',
  'MESH:D003367': 'Cotinine',
  'CHEBI:CHEBI:47034': '[8]annulene'},
 ['CHEBI:CHEBI:47034', 'HGNC:6860', 'MESH:D003367'])

In [40]:
excluded_longforms = []

In [41]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [42]:
additional_entities = {} #{'HGNC:6860': ['MAP3K8', ['MAP3K8', 'kinase kinase 8']]}

In [43]:
unambiguous_agent_texts = {}

In [44]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [27]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [28]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [29]:
intersection1

[('HGNC:6860', 'HGNC:6860', 118)]

In [30]:
intersection2

[('HGNC:6860', 'HGNC:6860', 0),
 ('ungrounded', 'HGNC:6860', 0),
 ('CHEBI:CHEBI:47034', 'HGNC:6860', 0),
 ('MESH:D003367', 'HGNC:6860', 0)]

In [31]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [32]:
names.update({key: value[0] for key, value in additional_entities.values()})
pos_labels = list(set(pos_labels) | additional_entities.keys())

In [45]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 01:03:59] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 01:04:00] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8666666666666666 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [46]:
classifier.stats

{'label_distribution': {'HGNC:6860': 10,
  'ungrounded': 19,
  'CHEBI:CHEBI:47034': 5,
  'MESH:D003367': 5},
 'f1': {'mean': 0.866667, 'std': 0.171594},
 'precision': {'mean': 0.85, 'std': 0.2},
 'recall': {'mean': 0.9, 'std': 0.122474},
 'CHEBI:CHEBI:47034': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'ungrounded': {'f1': {'mean': 0.949206, 'std': 0.063014},
  'pr': {'mean': 0.95, 'std': 0.1},
  'rc': {'mean': 0.96, 'std': 0.08}},
 'HGNC:6860': {'f1': {'mean': 0.933333, 'std': 0.133333},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.9, 'std': 0.2}},
 'MESH:D003367': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}}}

In [47]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [48]:
disamb.dump(model_name, results_path)

In [49]:
print(disamb.info())

Disambiguation model for COT

Produces the disambiguations:
	Cotinine*	MESH:D003367
	MAP3K8*	HGNC:6860
	[8]annulene*	CHEBI:CHEBI:47034

Class level metrics:
--------------------
Grounding  	Count	F1     
 Ungrounded	19	0.94921
     MAP3K8*	10	0.93333
[8]annulene*	 5	    0.8
   Cotinine*	 5	    0.8

Weighted Metrics:
-----------------
	F1 score:	0.86667
	Precision:	0.85
	Recall:		0.9

* Positive labels
See Docstring for explanation



In [50]:
model_to_s3(disamb)